In [1]:
import tensorflow as tf
import numpy as np
import pprint

In [2]:
batch_size = 1

num_classes = 3            #분류할 사전의 크기 

learning_rate = 0.01
sequence_length = 10         

output_dim = 3

In [3]:
X = tf.placeholder(tf.float32, [None, sequence_length,num_classes], name="X")
Y = tf.placeholder(tf.float32, [None, output_dim], name="Y")

In [4]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=num_classes, state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell]*2, state_is_tuple= True)

In [5]:
# outputs: unfolding size x hidden size, state = hidden size
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X,initial_state=initial_state,dtype=tf.float32)


#[ batch. X_dim*2],  [output_dim   , X_dim*4]
#[16,256], [129,512]. 

print(outputs)

Tensor("rnn/transpose_1:0", shape=(1, 10, 3), dtype=float32)


In [6]:
# FC layer
print(outputs)
print(outputs[:,-1])
#dense1 = tf.layers.dense(inputs=dense1_input, units=num_classes, activation=tf.nn.relu)
dense1 = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim, activation_fn=None)
dense2 = tf.layers.dense(inputs=dense1, units=output_dim, activation=tf.nn.relu)

print(dense1)
print(dense2)


Tensor("rnn/transpose_1:0", shape=(1, 10, 3), dtype=float32)
Tensor("strided_slice:0", shape=(1, 3), dtype=float32)
Tensor("fully_connected/BiasAdd:0", shape=(1, 3), dtype=float32)
Tensor("dense/Relu:0", shape=(1, 3), dtype=float32)


In [7]:
print(dense2)
print(Y)


logits= tf.layers.dense(inputs=dense2, units=output_dim)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
print(outputs)
print(Y)

Tensor("dense/Relu:0", shape=(1, 3), dtype=float32)
Tensor("Y:0", shape=(?, 3), dtype=float32)
Tensor("rnn/transpose_1:0", shape=(1, 10, 3), dtype=float32)
Tensor("Y:0", shape=(?, 3), dtype=float32)


In [8]:
dataX = [[[0, 2, 3],[11, 16, 1],[ 3, 4,1],[ 3, 4,1],[ 3, 4,1],
         [0, 2, 3],[11, 16, 1],[ 3, 4,1],[ 3, 4,1],[ 3, 4,1]]]

dataY = [[0, 0,1]]

In [9]:

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l,results = sess.run([optimizer, cost,logits], feed_dict={X: dataX, Y:dataY})
    print(_, l)

None 1.3202145
None 1.1911051
None 1.0741347
None 0.9675539
None 0.89521253
None 0.8478997
None 0.8006821
None 0.7537476
None 0.7073032
None 0.66156226
None 0.616733
None 0.5730027
None 0.5305229
None 0.48940274
None 0.44971102
None 0.41148666
None 0.3747552
None 0.33954692
None 0.3059113
None 0.27392372
None 0.2436848
None 0.21531172
None 0.18892491
None 0.16463135
None 0.1425097
None 0.12259681
None 0.10488001
None 0.08929625
None 0.07573645
None 0.06405443
None 0.054078992
None 0.045625843
None 0.038508262
None 0.032545883
None 0.02757017
None 0.023428988
None 0.019987365
None 0.017128514
None 0.014752467
None 0.012775247
None 0.011126379
None 0.009747763
None 0.008591357
None 0.0076180655
None 0.0067956285
None 0.006097761
None 0.005503147
None 0.0049944166
None 0.0045571043
None 0.0041796714
None 0.0038523285
None 0.0035675233
None 0.0033184006
None 0.0030996394
None 0.0029069807
None 0.0027363973
None 0.0025849282
None 0.002449847
None 0.0023290194
None 0.0022205475
None 0.002122

None 9.95348e-05
None 9.9058016e-05
None 9.8700424e-05
None 9.8342825e-05
None 9.7985234e-05
None 9.762764e-05
None 9.727005e-05
None 9.691246e-05
None 9.655486e-05
None 9.619727e-05
None 9.583968e-05
None 9.548208e-05
None 9.512449e-05
None 9.4766896e-05
None 9.44093e-05
None 9.4051706e-05
None 9.3694114e-05
None 9.3336515e-05
None 9.2978924e-05
None 9.2621325e-05
None 9.238293e-05
None 9.202534e-05
None 9.1667745e-05
None 9.131015e-05
None 9.0952555e-05
None 9.071415e-05
None 9.035656e-05
None 8.999896e-05
None 8.964137e-05
None 8.940297e-05
None 8.9045374e-05
None 8.8687775e-05
None 8.844938e-05
None 8.809179e-05
None 8.773419e-05
None 8.749579e-05
None 8.713819e-05
None 8.6899796e-05
None 8.65422e-05
None 8.6184606e-05
None 8.59462e-05
None 8.558861e-05
None 8.5350206e-05
None 8.4992615e-05
None 8.475421e-05
None 8.439662e-05


In [10]:
y_true = sess.run(tf.argmax(dataY,1))
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: dataX })

In [11]:
y_pred

array([2], dtype=int64)

In [13]:
results

array([[-6.9781017, -4.37972  ,  5.071638 ]], dtype=float32)